! alle Pakete installieren

Therapie-Warteschlage Nordbayern – SimPy-Modell (DES)

Idee:
- Entities  : Patient*innen
- Resource  : Therapieplätze (Kapazität = Anzahl paralleler Therapien)
- Zeitmaß   : Wochen
- Ankunft   : Exponentiell verteilte Zwischenankunftszeiten (Poisson-Prozess)
- Service   : Therapiedauer ~ Gleichverteilung [MIN_SERVICE_WEEKS, MAX_SERVICE_WEEKS]

# Pakete importieren 

In [2]:
import simpy
import numpy as np
import statistics as stats

# Code

## 1) Parameter des Systems - Szenario einstellen

In [3]:
SEED = 42

# Nachfrage / Anmeldungen
ARRIVAL_RATE_PER_WEEK = 15.0   # Ø neue Anmeldungen pro Woche (λ)
MEAN_INTERARRIVAL = 1.0 / ARRIVAL_RATE_PER_WEEK  # mittlere Zeit zw. Ankünften (Wochen)

N_PATIENTS = 2000              # wie viele Patient*innen simuliert werden sollen

# Therapie-Kapazität
N_THERAPEUTEN = 20             # Anzahl Therapeut*innen
PLÄTZE_PRO_THERAPEUT = 8       # wie viele parallele Fälle pro Person (Fallzahl / Jahr etc.)
THERAPIE_CAPACITY = N_THERAPEUTEN * PLÄTZE_PRO_THERAPEUT

# Therapiedauer (Servicezeit) in Wochen
MIN_SERVICE_WEEKS = 10.0       # z. B. 10 Sitzungen à 1 Woche
MAX_SERVICE_WEEKS = 40.0       # z. B. 40 Sitzungen


Globale Listen für Auswertung

In [8]:
wait_times = []        # reine Wartezeit von Anmeldung bis Therapiebeginn
system_times = []      # Zeit im System: Anmeldung bis Therapieende
service_times = []     # tatsächliche Therapiedauer

# für Auslastung: Summe aller Servicezeiten (über alle Plätze)
total_service_time_all = 0.0

## 2) Funktionen

Patient*in

In [4]:
def patient_process(env, name, therapy_resource, rng):
    """
    Prozess, den eine Patient*in durchläuft:
    1) Ankunft
    2) Warten auf freien Therapieplatz
    3) Therapie (Servicezeit)
    4) Verlassen des Systems
    """
    global total_service_time_all

    t_arrival = env.now  # Ankunftszeit in Wochen

    # 1. Therapieplatz anfordern
    with therapy_resource.request() as req:
        # Warten, bis Platz frei wird
        yield req

        t_start = env.now
        wait = t_start - t_arrival
        wait_times.append(wait)

        # 2. Zufällige Therapiedauer ziehen
        service_duration = rng.uniform(MIN_SERVICE_WEEKS, MAX_SERVICE_WEEKS)
        service_times.append(service_duration)
        total_service_time_all += service_duration

        # 3. Therapie „läuft“
        yield env.timeout(service_duration)

        t_end = env.now
        system_times.append(t_end - t_arrival)
        # Prozess endet automatisch, Ressource wird beim Verlassen des with-Blocks freigegeben


## 3) Prozess: Generator für neue Patient*innen

In [5]:
def arrival_process(env, therapy_resource, rng):
    """
    Erzeugt nach und nach neue Patient*innen mit zufälligen Zwischenankunftszeiten.
    """
    for i in range(N_PATIENTS):
        # Zeit bis zur nächsten Anmeldung (Exponentialverteilung)
        interarrival = rng.exponential(scale=MEAN_INTERARRIVAL)
        yield env.timeout(interarrival)

        env.process(
            patient_process(env, f"Patient {i}", therapy_resource, rng)
        )


## 4) Simulationslauf

In [6]:
def run_simulation():
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()

    therapy_resource = simpy.Resource(env, capacity=THERAPIE_CAPACITY)

    env.process(arrival_process(env, therapy_resource, rng))

    # Ohne "until": Simulation läuft, bis alle Patienten fertig sind
    env.run()

    sim_time = env.now  # letzte Zeit in der Simulation (in Wochen)


## 5) Kennzahlen berechnen 

In [9]:
if not wait_times:
        print("Keine Patient*innen simuliert – überprüfe N_PATIENTS.")
        return

    # Wartezeiten
mean_wait = stats.mean(wait_times)
median_wait = np.quantile(wait_times, 0.5)
q75_wait = np.quantile(wait_times, 0.75)
q90_wait = np.quantile(wait_times, 0.90)

share_over_12 = np.mean(np.array(wait_times) > 12.0)
share_over_24 = np.mean(np.array(wait_times) > 24.0)

    # Systemzeit
mean_system = stats.mean(system_times)

    # Auslastung: Summe aller Servicezeiten / (Kapazität * Simulationszeit)
utilization = total_service_time_all / (THERAPIE_CAPACITY * sim_time)

SyntaxError: 'return' outside function (1069528379.py, line 3)

## 6) Ergbenisse ausgeben

In [ ]:
print("=== Simulation Therapie-Warteschlange (DES / SimPy) ===")
print(f"Anzahl Therapeut*innen:              {N_THERAPEUTEN}")
print(f"Plätze pro Therapeut*in:             {PLÄTZE_PRO_THERAPEUT}")
print(f"Gesamtkapazität (Parallele Fälle):   {THERAPIE_CAPACITY}")
print(f"Simulierte Patient*innen:            {N_PATIENTS}")
print(f"Simulationszeit (letztes Event):     {sim_time:.1f} Wochen\n")

print("Wartezeit bis Therapiebeginn (Wochen)")
print(f"  Mittelwert:                        {mean_wait:.2f}")
print(f"  Median:                            {median_wait:.2f}")
print(f"  75%-Quantil:                       {q75_wait:.2f}")
print(f"  90%-Quantil:                       {q90_wait:.2f}")
print(f"  Anteil > 12 Wochen:                {share_over_12*100:.1f} %")
print(f"  Anteil > 24 Wochen:                {share_over_24*100:.1f} %\n")

print("Gesamtzeit im System (Anmeldung bis Therapieende)")
print(f"  Mittelwert:                        {mean_system:.2f} Wochen\n")

print("Auslastung der Therapie-Kapazität")
print(f"  Geschätzte Auslastung ρ:           {utilization*100:.1f} %")


if __name__ == "__main__":
    run_simulation()

# Komplett

In [10]:
import simpy
import numpy as np
import statistics as stats

# ---------------------------------------------------------
# Parameter
# ---------------------------------------------------------

SEED = 42
ARRIVAL_RATE_PER_WEEK = 15.0
MEAN_INTERARRIVAL = 1.0 / ARRIVAL_RATE_PER_WEEK
N_PATIENTS = 2000

N_THERAPEUTEN = 20
PLAETZE_PRO_THERAPEUT = 8
THERAPIE_CAPACITY = N_THERAPEUTEN * PLAETZE_PRO_THERAPEUT

MIN_SERVICE_WEEKS = 10.0
MAX_SERVICE_WEEKS = 40.0

# ---------------------------------------------------------
# Speicher für Auswertung
# ---------------------------------------------------------

wait_times = []
system_times = []
service_times = []
total_service_time_all = 0.0


# ---------------------------------------------------------
# Prozesse
# ---------------------------------------------------------

def patient_process(env, name, therapy_resource, rng):
    global total_service_time_all

    t_arrival = env.now

    # warten auf Ressource
    with therapy_resource.request() as req:
        yield req
        t_start = env.now

        wait = t_start - t_arrival
        wait_times.append(wait)

        # Servicezeit ziehen
        service_duration = rng.uniform(MIN_SERVICE_WEEKS, MAX_SERVICE_WEEKS)
        service_times.append(service_duration)
        total_service_time_all += service_duration

        yield env.timeout(service_duration)

        t_end = env.now
        system_times.append(t_end - t_arrival)


def arrival_process(env, therapy_resource, rng):
    for i in range(N_PATIENTS):
        interarrival = rng.exponential(scale=MEAN_INTERARRIVAL)
        yield env.timeout(interarrival)
        env.process(patient_process(env, f"Patient {i}", therapy_resource, rng))


# ---------------------------------------------------------
# Hauptfunktion
# ---------------------------------------------------------

def run_simulation():
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()

    therapy_resource = simpy.Resource(env, capacity=THERAPIE_CAPACITY)
    env.process(arrival_process(env, therapy_resource, rng))

    env.run()
    sim_time = env.now

    # --- Kennzahlen ---
    if not wait_times:
        print("Keine Patient*innen simuliert – prüfe N_PATIENTS.")
        return

    mean_wait = stats.mean(wait_times)
    median_wait = np.quantile(wait_times, 0.5)
    q75_wait = np.quantile(wait_times, 0.75)
    q90_wait = np.quantile(wait_times, 0.90)

    share_over_12 = np.mean(np.array(wait_times) > 12)
    share_over_24 = np.mean(np.array(wait_times) > 24)

    mean_system = stats.mean(system_times)

    utilization = total_service_time_all / (THERAPIE_CAPACITY * sim_time)

    # --- Ausgabe ---
    print("=== Simulation Therapie-Warteschlange ===")
    print(f"Gesamtkapazität:                {THERAPIE_CAPACITY}")
    print(f"Simulierte Patient*innen:       {N_PATIENTS}")
    print(f"Letzte Simulationszeit:         {sim_time:.1f} Wochen\n")

    print("Wartezeit (Wochen):")
    print(f"  Mittelwert:                   {mean_wait:.2f}")
    print(f"  Median:                       {median_wait:.2f}")
    print(f"  75%-Quantil:                  {q75_wait:.2f}")
    print(f"  90%-Quantil:                  {q90_wait:.2f}")
    print(f"  Anteil > 12 Wochen:           {share_over_12*100:.1f}%")
    print(f"  Anteil > 24 Wochen:           {share_over_24*100:.1f}%\n")

    print("Systemzeit (Anmeldung → Therapieende):")
    print(f"  Mittelwert:                   {mean_system:.2f}\n")

    print("Auslastung:")
    print(f"  ρ =                           {utilization*100:.1f} %")


# ---------------------------------------------------------
# Simulation ausführen
# ---------------------------------------------------------

run_simulation()

=== Simulation Therapie-Warteschlange ===
Gesamtkapazität:                160
Simulierte Patient*innen:       2000
Letzte Simulationszeit:         338.2 Wochen

Wartezeit (Wochen):
  Mittelwert:                   75.05
  Median:                       75.10
  75%-Quantil:                  116.80
  90%-Quantil:                  143.57
  Anteil > 12 Wochen:           86.6%
  Anteil > 24 Wochen:           79.3%

Systemzeit (Anmeldung → Therapieende):
  Mittelwert:                   99.78

Auslastung:
  ρ =                           91.4 %


## a)